# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1450748903 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Users\abautista\AppData\Local\Dato\Dato Launcher\lib\site-packages\certifi\cacert.pem
1450748903 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to alejandro.bautista@infor.com and will expire on September 30, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-7080 - Server binary: C:\Users\abautista\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\ABAUTI~1\AppData\Local\Temp\graphlab_server_1450748903.log.0
[INFO] GraphLab Server Version: 1.7.1


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [4]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: Number of coefficients    : 4
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 1.008798     | 4146407.600631     | 258679.804477 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | 87910.0724924  |
| sqft_living |  None | 315.403440552  |
|   bedrooms  |  None | -65080.2155528 |
|  bathrooms  |  None | 6944.02019265  |
+-------------+-------+----------------+
[4 rows x 3 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [31]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878
print train_data['price']
print example_predictions[1]

271789.505878
[221900.0, 538000.0, 180000.0, 604000.0, 510000.0, 1225000.0, 257500.0, 291850.0, 229500.0, 323000.0, 662500.0, 468000.0, 400000.0, 530000.0, 395000.0, 485000.0, 189000.0, 230000.0, 385000.0, 2000000.0, 285000.0, 252700.0, 329000.0, 937000.0, 667000.0, 438000.0, 719000.0, 280000.0, 687500.0, 322500.0, 696000.0, 550000.0, 640000.0, 240000.0, 625000.0, 861990.0, 685000.0, 309000.0, 488000.0, 210490.0, 785000.0, 450000.0, 1350000.0, 228000.0, 345000.0, 600000.0, 585000.0, 920000.0, 885000.0, 301000.0, 951000.0, 430000.0, 650000.0, 505000.0, 549000.0, 425000.0, 317625.0, 975000.0, 287000.0, 204000.0, 1325000.0, 1040000.0, 325000.0, 360000.0, 832500.0, 380000.0, 480000.0, 410000.0, 720000.0, 390000.0, 355000.0, 356000.0, 315000.0, 940000.0, 305000.0, 461000.0, 215000.0, 335000.0, 1099880.0, 153000.0, 430000.0, 700000.0, 905000.0, 199000.0, 314000.0, 437500.0, 850830.0, 555000.0, 699950.0, 1088000.0, 290000.0, 375000.0, 460000.0, 188500.0, 680000.0, 597750.0, 570000.0, 272500.0

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [29]:
def get_residual_sum_of_squares(model, data, outcome):
    
    RSS         = 0
    predictions = 0
    
    # First get the predictions
    predictions = model.predict(test_data)
    
    # Then compute the residuals/errors

    for i in range(len(data)):
        RSS = RSS + (outcome[i] - predictions[i])**2
   
    
    # Then square and add them up

    
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [30]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


In [10]:
test_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980,2,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553,2,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186,1,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495,2,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400,1,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807,1.5,0


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [32]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [93]:
#bedrooms_squared
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)
print round(test_data['bedrooms_squared'].mean(),2)

#bed_bath_rooms
#print train_data['bedrooms','bathrooms'] call two columns
train_data['bed_bath_rooms']= train_data['bedrooms']*train_data['bathrooms'] #multiply two columns
test_data['bed_bath_rooms']= test_data['bedrooms']*test_data['bathrooms'] #multiply two columns
print round(test_data['bed_bath_rooms'].mean(),2)

12.45
7.5


In [94]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
print round(test_data['log_sqft_living'].mean(),2)

#lat_plus_long
train_data['lat_plus_long']= train_data['lat']+train_data['long'] #sum two columns
test_data['lat_plus_long']= test_data['lat']+test_data['long'] #sum two columns
print round(test_data['lat_plus_long'].mean(),2)

7.55
-74.65


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [95]:
print round(test_data['bedrooms_squared'].mean(),2)
print round(test_data['bed_bath_rooms'].mean(),2)
print round(test_data['log_sqft_living'].mean(),2)
print round(test_data['lat_plus_long'].mean(),2)

12.45
7.5
7.55
-74.65


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [77]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [79]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data,target='price',features=model_1_features,validation_set=None)
model_2 = graphlab.linear_regression.create(train_data,target='price',features=model_2_features,validation_set=None)
model_3 = graphlab.linear_regression.create(train_data,target='price',features=model_3_features,validation_set=None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 5
PROGRESS: Number of unpacked features : 5
PROGRESS: Number of coefficients    : 6
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.011001     | 4074878.213096     | 236378.596455 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROG

In [83]:
# Examine/extract each model's coefficients:
print model_1.get("coefficients")
print model_2.get("coefficients")
print model_3.get("coefficients")

+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | -56140675.7444 |
| sqft_living |  None | 310.263325778  |
|   bedrooms  |  None | -59577.1160682 |
|  bathrooms  |  None | 13811.8405418  |
|     lat     |  None | 629865.789485  |
|     long    |  None | -214790.285186 |
+-------------+-------+----------------+
[6 rows x 3 columns]

+----------------+-------+----------------+
|      name      | index |     value      |
+----------------+-------+----------------+
|  (intercept)   |  None | -54410676.1152 |
|  sqft_living   |  None | 304.449298057  |
|    bedrooms    |  None | -116366.043231 |
|   bathrooms    |  None | -77972.3305135 |
|      lat       |  None | 625433.834953  |
|      long      |  None | -203958.60296  |
| bed_bath_rooms |  None | 26961.6249092  |
+----------------+-------+----------------+
[7 rows x 3 columns]

+------------------+-------+----------------+
|       name    

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

### In model 1, the weight for bathroom is positive whereas in model 2, the weight for bathroom is negative and this is due to the fact that the bed_bathh_rooms variable was added.  

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [89]:
def get_residual_sum_of_squares_train_data(model, data, outcome):
    
    RSS         = 0
    predictions = 0
    
    # First get the predictions
    predictions = model.predict(train_data)
    
    # Then compute the residuals/errors

    for i in range(len(data)):
        RSS = RSS + (outcome[i] - predictions[i])**2
   
    
    # Then square and add them up

    
    return(RSS)  

In [91]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model_1_train = get_residual_sum_of_squares_train_data(model_1, train_data, train_data['price'])
rss_model_2_train = get_residual_sum_of_squares_train_data(model_2, train_data, train_data['price'])
rss_model_3_train = get_residual_sum_of_squares_train_data(model_3, train_data, train_data['price'])
print rss_model_1_train 
print rss_model_2_train 
print rss_model_3_train

9.71328233544e+14
9.61592067856e+14
9.05276314555e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [87]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1 = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2 = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3 = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print rss_model_1 
print rss_model_2 
print rss_model_3 

2.26568089093e+14
2.10672843066e+16
2.28659096444e+15


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.